In [2]:
from load_data import load_train_data
import pandas as pd

In [3]:
train_data= load_train_data()
train_data.head(5)

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation


In [4]:
print(f"학습데이터 갯수: {len(train_data)}\n중복 제거 후 데이터 갯수: {train_data['sentence'].nunique()}")

학습데이터 갯수: 32470
중복 제거 후 데이터 갯수: 28803


In [5]:
duplicate_data_first = train_data[train_data.duplicated(subset=['sentence'], keep='first')]
duplicate_data_last = train_data[train_data.duplicated(subset=['sentence'], keep='last')]
duplicate_data_first = duplicate_data_first.sort_values(by='sentence')
duplicate_data_first.head(5)

,id,sentence,subject_entity,object_entity,label
14736,14736,"""2006년 지방선거에 참패하고 민주당과 그 전신인 열린 우리당은 노 씨의 국정 운...",'열린 우리당','정세균',no_relation
28051,28051,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...",'소이현','인교진',no_relation
32434,32434,"""탕약망""이란 예수회 선교사 천문학자 아담 샬의 중국 이름이다.",'아담 샬','예수회',per:employee_of
29166,29166,"'2019 코리아 스타트업 테크쇼'는 경상남도, 창원시, 경남지방중소벤처기업청이 공...",'창원시','경상남도',org:place_of_headquarters
23316,23316,"'Project Fairy(프로젝트 페어리)'라는 유닛명으로 가나하 히비키, 시죠 ...",'가나하 히비키','프로젝트 페어리',per:employee_of


In [6]:
duplicate_data_last = duplicate_data_last.sort_values(by='sentence')
duplicate_data_last.head(5)

,id,sentence,subject_entity,object_entity,label
919,919,"""2006년 지방선거에 참패하고 민주당과 그 전신인 열린 우리당은 노 씨의 국정 운...",'열린 우리당','노무현',no_relation
3588,3588,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...",'인교진','소이현',no_relation
19427,19427,"""탕약망""이란 예수회 선교사 천문학자 아담 샬의 중국 이름이다.",'탕약망','예수회',per:employee_of
11401,11401,"'2019 코리아 스타트업 테크쇼'는 경상남도, 창원시, 경남지방중소벤처기업청이 공...",'경상남도','창원시',org:members
16566,16566,"'Project Fairy(프로젝트 페어리)'라는 유닛명으로 가나하 히비키, 시죠 ...",'시죠 타카네','프로젝트 페어리',per:employee_of


# 중복데이터
- 문장기분 중복데이터가 존재하며 몇몇의 경우 라벨이 아에 다르다.
- 아마 sub와 obj의 관계가 달라서 생기는 문제인듯 하다.

In [7]:
dup_data = pd.DataFrame({
  'sentence': list(duplicate_data_first['sentence']),
  'label_f': list(duplicate_data_first['label']),
  'label_l': list(duplicate_data_last['label'])
  })
dup_data.head(10)

,sentence,label_f,label_l
0,"""2006년 지방선거에 참패하고 민주당과 그 전신인 열린 우리당은 노 씨의 국정 운...",no_relation,no_relation
1,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...",no_relation,no_relation
2,"""탕약망""이란 예수회 선교사 천문학자 아담 샬의 중국 이름이다.",per:employee_of,per:employee_of
3,"'2019 코리아 스타트업 테크쇼'는 경상남도, 창원시, 경남지방중소벤처기업청이 공...",org:place_of_headquarters,org:members
4,"'Project Fairy(프로젝트 페어리)'라는 유닛명으로 가나하 히비키, 시죠 ...",per:employee_of,per:employee_of
5,'노래가 좋아'는 아나운서 도경완 씨와 장윤정 씨 부부가 함께 MC를 보는 방송이다.,per:spouse,per:spouse
6,'대리게임' 논란으로 비판을 받았던 정의당 비례대표 1번 류호정 후보가 입장을 밝혔다.,per:title,per:employee_of
7,"'미스터트롯'에는 가수 천명훈 씨, 이투스 강사 정승제 씨, Y2K 출신 가수 고재...",per:employee_of,per:employee_of
8,'삼성 X파일 사건'은 1997년 이학수 전 삼성 부회장과 중앙일보 홍석현 회장이 ...,org:alternate_names,org:alternate_names
9,'좌파당/민주사회당'과 '선거연대' 사이의 통합 협상은 계속되다가 마침내 2007년...,org:alternate_names,no_relation


In [8]:
from load_data import tokenized_dataset

In [9]:
from transformers import AutoTokenizer

MODEL_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenized_train = tokenized_dataset(train_data, tokenizer)

# 데이터 입력방식
- [CLS] sub [SEP] obj [SEP] 문장
- [CLS] 문장 [SEP] sub [SEP] obj 식으로 하면 성능향상이 있을까?
- token_type_ids 는 두번째 문장에만 1이 표시되도록 설계됨

In [10]:
tokenized_train.token_type_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [11]:
tokenizer.decode(tokenized_train.input_ids[0])

"[CLS]'비틀즈'[SEP]'조지 해리슨'[SEP] 〈 Something 〉 는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《 Abbey Road 》 에 담은 노래다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [12]:
from transformers import AutoConfig, AutoModelForSequenceClassification

model_config =  AutoConfig.from_pretrained(MODEL_NAME)
model_config.num_labels = 30

model =  AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=model_config)
model

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [13]:
print(model.config)

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForPretraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
 

In [14]:
train_data_copy = train_data.copy()
train_data_copy.head(5)

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation


In [15]:
train_data_copy = train_data_copy.rename(columns={"subject_entity": "object_entity", "object_entity": "subject_entity"})
train_data_copy.head(5)

,id,sentence,object_entity,subject_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation


In [16]:
label_list = ['no_relation', 'org:members', 'org:alternate_names',
  'per:children', 'per:alternate_names',
  'per:other_family', 'per:colleagues', 'per:siblings',
  'per:spouse', 'org:member_of', 'per:parents']

train_data_copy = train_data_copy[train_data_copy.label.isin(label_list)]
train_data_copy[train_data_copy.label == "org:members"].head(5)

,id,sentence,object_entity,subject_entity,label
13,13,뉴질랜드 1차산업부 생물보안 대변인 캐서린 더시 박사는 과일파리를 잡기 위해 오클랜...,'뉴질랜드','오클랜드',org:members
47,47,"반민특위는 조사를 담당하는 특별조사위원회, 기소 및 송치 업무를 담당하는 특별검찰,...",'반민특위','특별조사위원회',org:members
135,135,"한편, 러시아의 경우에도 러시아 측의 조치로 인해 올 3월부터 인천-모스크바 간 정...",'러시아','모스크바',org:members
286,286,한편 경상남도의 저상버스 도입과 관련하여 창원시에서는 적극적으로 추진하려 했지만 마...,'경상남도','창원시',org:members
326,326,그는 “현대모비스는 약점으로 지적되어 왔던 높은 현대차 그룹 의존도 점차 낮아지고 ...,'현대차 그룹','현대모비스',org:members


In [17]:
change_values = { "org:member_of": "org:members", "per:parents": "per:children", "org:members": "org:member_of", "per:children": "per:parents" }
train_data_copy2 = train_data_copy.replace({ "label": change_values })
train_data_copy2[train_data_copy2.label == "org:members"].head(5)

,id,sentence,object_entity,subject_entity,label
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:members
17,17,제2총군은 태평양 전쟁 말기에 일본 본토에 상륙하려는 연합군에게 대항하기 위해 설립...,'제2총군','일본 제국 육군',org:members
23,23,스포츠계에서도 막강한 영향력을 펼치고 있어 2008년 9월 만수르 빈 자이드 알나하...,'맨체스터 시티 FC','프리미어리그',org:members
42,42,"2008-09 시즌, 그는 페페의 10경기 출장 정지 징계로부터 수혜를 받은 선수로...",'세비야','스페인',org:members
43,43,대한민국에서는 2008년 대원미디어㈜에서 수입하여 자사인 대원방송㈜에서 더빙 작업을...,'대원방송','대원미디어',org:members


In [18]:
train_data_copy2[train_data_copy2.label == "org:member_of"].head(5)

,id,sentence,object_entity,subject_entity,label
13,13,뉴질랜드 1차산업부 생물보안 대변인 캐서린 더시 박사는 과일파리를 잡기 위해 오클랜...,'뉴질랜드','오클랜드',org:member_of
47,47,"반민특위는 조사를 담당하는 특별조사위원회, 기소 및 송치 업무를 담당하는 특별검찰,...",'반민특위','특별조사위원회',org:member_of
135,135,"한편, 러시아의 경우에도 러시아 측의 조치로 인해 올 3월부터 인천-모스크바 간 정...",'러시아','모스크바',org:member_of
286,286,한편 경상남도의 저상버스 도입과 관련하여 창원시에서는 적극적으로 추진하려 했지만 마...,'경상남도','창원시',org:member_of
326,326,그는 “현대모비스는 약점으로 지적되어 왔던 높은 현대차 그룹 의존도 점차 낮아지고 ...,'현대차 그룹','현대모비스',org:member_of


In [19]:
cols = ['id', 'sentence', 'subject_entity', 'object_entity', 'label']
train_data_copy2 = train_data_copy2[cols]
train_data_copy2[train_data_copy2.label == "org:members"].head(5)

,id,sentence,subject_entity,object_entity,label
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'한국프로축구연맹','광주FC',org:members
17,17,제2총군은 태평양 전쟁 말기에 일본 본토에 상륙하려는 연합군에게 대항하기 위해 설립...,'일본 제국 육군','제2총군',org:members
23,23,스포츠계에서도 막강한 영향력을 펼치고 있어 2008년 9월 만수르 빈 자이드 알나하...,'프리미어리그','맨체스터 시티 FC',org:members
42,42,"2008-09 시즌, 그는 페페의 10경기 출장 정지 징계로부터 수혜를 받은 선수로...",'스페인','세비야',org:members
43,43,대한민국에서는 2008년 대원미디어㈜에서 수입하여 자사인 대원방송㈜에서 더빙 작업을...,'대원미디어','대원방송',org:members


In [20]:
print(len(train_data_copy2), len(train_data))
k = train_data.append(train_data_copy2)

16620 32470


In [24]:
k_first = k[k.duplicated(subset=['sentence', 'subject_entity', 'object_entity'], keep='first')]
k_last = k[k.duplicated(subset=['sentence', 'subject_entity', 'object_entity'], keep='last')]
k_first = k_first.sort_values(by='sentence')
k_last = k_last.sort_values(by='sentence')

In [25]:
k_first.head(5)

,id,sentence,subject_entity,object_entity,label
3588,3588,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...",'소이현','인교진',no_relation
28051,28051,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...",'인교진','소이현',no_relation
11401,11401,"'2019 코리아 스타트업 테크쇼'는 경상남도, 창원시, 경남지방중소벤처기업청이 공...",'창원시','경상남도',org:member_of
19742,19742,'노래가 좋아'는 아나운서 도경완 씨와 장윤정 씨 부부가 함께 MC를 보는 방송이다.,'장윤정','도경완',per:spouse
31655,31655,'노래가 좋아'는 아나운서 도경완 씨와 장윤정 씨 부부가 함께 MC를 보는 방송이다.,'도경완','장윤정',per:spouse


In [26]:
k_last.head(5)

,id,sentence,subject_entity,object_entity,label
28051,28051,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...",'소이현','인교진',no_relation
3588,3588,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...",'인교진','소이현',no_relation
29166,29166,"'2019 코리아 스타트업 테크쇼'는 경상남도, 창원시, 경남지방중소벤처기업청이 공...",'창원시','경상남도',org:place_of_headquarters
31655,31655,'노래가 좋아'는 아나운서 도경완 씨와 장윤정 씨 부부가 함께 MC를 보는 방송이다.,'장윤정','도경완',per:spouse
19742,19742,'노래가 좋아'는 아나운서 도경완 씨와 장윤정 씨 부부가 함께 MC를 보는 방송이다.,'도경완','장윤정',per:spouse


KeyError: 0